In [ ]:
! pip install transformers datasets huggingface-hub
! pip install pandas
! pip install numpy

# NLP - RNNs, Transformers, Hugging Face

In this notebook, we will be understanding and delving more deeply into NLP. Currently, NLP is the most popular application of deep learning. All Large Language Models (LLMs) currently operate based on the transformer architecture to provide generative capabilities.

We will try to make a NLP classification model that can identify Charles Dickens' writings.

This notebook is based off of module 4 of Fast AI's on NLP.

### Language Models

A language model is a model that is trained to predict the next word in a text based on the previous words. A language model uses something called self-supervised training to achieve this. Without external labels, it can find labels within the text it needs to evaluate. To achieve this, our language model needs to develop a certain understanding of the language. This means that for our applications, our language model needs to understand the English language, the French language, the German language, etc.

An example of an application development process is the IMDb review classifier. We will use a language model that was trained on Wikipedia data. Unfortunately, this model might not be entirely suitable for IMDb review English. Wikipedia articles are usually written in a different style and format from an IMDb review. In order to get accurate classifications, we ought to fine-tune our model on IMDb English. From that fine-tuned model, we can then work on developing a classification model for IMDb movie reviews that will be very accurate.

The preceding process is called the Universal Language Model Fine-Tuning Process (ULMFit).

#### Recurrent Neural Networks - RNNs

Recurrent Neural Networks (RNNS) are a type of neural network architecture trained on sequential or time series data that are used to make machine learning models that can make sequential predictions using previous sequence elements as inputs for the predictions. RNNs use a hidden state that helps keep track of previous inputs. This is the recurrent part of the RNN. 

RNNS use a encoder-decoder model. This model is best explained by the following image: 

![encoder/decoder model](./resources/encoder-decoder.png)

For more information: 
- [IBM article](https://www.ibm.com/think/topics/recurrent-neural-networks)
- [Blog post by Zhaozhen Xu](https://www.baeldung.com/cs/rnns-transformers-nlp)

### Transformers

Transformers are a type of neural network architecture that is very capable of processing natural language. Unlike RNNs, transformers do not use any recurrence or have hidden states. This means they do not operate sequentially (ie, they do not need to go through each input one at a time). They use something called self-attention. Self-attention allows the model to weigh the importance of different input tokens when making predictions. Transformers consist of encoder and decoder layers, employing multi-head self-attention mechanisms and feedback neural networks. Thanks to these features, they are able to parallelize their operations and are faster.

Here is an image illustrating the transformer model.

![transformer model](./resources/transformer.png)

The transformer model was layed out in the 2017 seminal paper [*Attention Is All You Need*](./resources/attention.pdf).

### Tokenization and Numericalization

Our neural networks need to take in numbers as their inputs. We need to convert our sentences into numbers, there are two steps: 

- *Tokenization*: we split text up into tokens
- *Numericalization*: convert each token into a number

This process is model dependent. Each model will have a tokenizer associated with it. We'll see when developing our Dickens classifier. We'll use the "Deberta v3-small" model developed by Microsoft. 

In [ ]:
model_nm = "microsoft/deberta-v2-xlarge-mnli"

```AutoTokenizer``` is a HuggingFace Transformers class that allows us to get our tokenizer function.

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')

In [ ]:
tokz.tokenize("Hi! I am Sami")

### Our Dataset

In [ ]:
from datasets import load_dataset

dickens_ds = load_dataset("GuillermoTBB/charles-dickens-text-classification")
dickens_ds

In [ ]:
train, test = dickens_ds['train'], dickens_ds['test']

In [ ]:
train_df = train.to_pandas()
train_df

In [ ]:
def tok_func(ds):
    return tokz(ds['text'])

In [ ]:
tok_ds = train.map(tok_func, batched=True)
tok_ds

In [ ]:
import numpy as np
from numpy.random import normal, seed, uniform
np.random.seed(42)

dds = tok_ds.train_test_split(0.25, seed=42)
dds

### Training our model

In [ ]:
from transformers import TrainingArguments, Trainer

bs = 128
epochs = 4
learning_rate = 8e-5

In [ ]:
args = TrainingArguments('outputs', 
                         learning_rate = learning_rate, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine',
                         fp16=False,
                         eval_strategy='epoch', 
                         per_device_train_batch_size=bs, 
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs, 
                         weight_decay=0.01, 
                         report_to='none'
      )

#### Defining our metrics

We need to define a function to evaluate our model.

In [ ]:
def accuracy(prediction, label):
    return np.mean(prediction == label)

In [ ]:
def metrics(dataset):
    return {'accuracy': accuracy(*dataset)}

#### Training our model

The Transformers library provides some APIs to facilitate training.

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained(model_nm)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz,
                  compute_metrics=metrics
                  )

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(test)
preds